# ICS5110: Applied Machine Learning

Peter Galea St. John: 288098m\
Christian A. Vella


In [2]:
# Import some necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

######-ANN
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler


######-RF

from sklearn.tree import DecisionTreeClassifier

# Data Preparation

In [3]:
 with open('Cleaned-Data.csv') as csv_file:
    data = list(csv.reader(csv_file, delimiter='\t'))
    r = len(data)

In [4]:
df = pd.read_csv('Cleaned-Data.csv',na_values="?" )

In [5]:
df.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
0,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,0,1,China
1,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,1,0,China
2,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,1,0,0,China
3,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,0,1,China
4,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,1,0,China


In [6]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

,Country
0,China
1,China
2,China
3,China
4,China


In [7]:
obj_df["Country"].value_counts()

Italy                 31680
Other                 31680
China                 31680
Republic of Korean    31680
Spain                 31680
France                31680
Germany               31680
Iran                  31680
UAE                   31680
Other-EUR             31680
Name: Country, dtype: int64

In [8]:
cleanup_Countries = {"Country": {"China":0, "UAE":1, "Iran":2, "France":3, "Republic of Korean":4, "Spain":5, "Italy":6, "Germany":7, "Other-EUR":8, "Other":9}}

In [9]:
obj_df = obj_df.replace(cleanup_Countries)

In [10]:
obj_df.head()

,Country
0,0
1,0
2,0
3,0
4,0


In [11]:
df["Country"] = obj_df

In [12]:
severity_columns = df.filter(like='Severity_').columns
gender_columns = df.filter(like='Gender_').columns

df['Severity_None'].replace({1:'0',0:'No'},inplace =True)
df['Severity_Mild'].replace({1:'1',0:'No'},inplace =True)
df['Severity_Moderate'].replace({1:'2',0:'No'},inplace =True)
df['Severity_Severe'].replace({1:'3',0:'No'},inplace =True)

df['Condition'] = df[severity_columns].values.tolist()

df.drop(columns=['Severity_None','Severity_Mild','Severity_Moderate','Severity_Severe'])

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Contact_Dont-Know,Contact_No,Contact_Yes,Country,Condition
0,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,0,1,0,"[1, No, No, No]"
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,1,0,0,"[1, No, No, No]"
2,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,1,0,0,0,"[1, No, No, No]"
3,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,0,1,0,"[No, 2, No, No]"
4,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,1,0,0,"[No, 2, No, No]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316795,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,1,0,9,"[No, No, No, 3]"
316796,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,1,0,0,9,"[No, No, No, 3]"
316797,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,9,"[No, No, 0, No]"
316798,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,1,0,9,"[No, No, 0, No]"


In [13]:
def removing(list1):
    list1 = set(list1) 
    list1.discard("No")
    a = ''.join(list1)
    return a

In [14]:
df['Condition'] = df['Condition'].apply(removing)

In [15]:
df.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country,Condition
0,1,1,1,1,1,0,1,1,1,1,...,0,1,No,No,No,0,0,1,0,1
1,1,1,1,1,1,0,1,1,1,1,...,0,1,No,No,No,0,1,0,0,1
2,1,1,1,1,1,0,1,1,1,1,...,0,1,No,No,No,1,0,0,0,1
3,1,1,1,1,1,0,1,1,1,1,...,0,No,2,No,No,0,0,1,0,2
4,1,1,1,1,1,0,1,1,1,1,...,0,No,2,No,No,0,1,0,0,2


In [16]:
headers= df.columns.tolist()

In [17]:
X= pd.DataFrame.to_numpy(df.drop(['Condition'],axis=1))
y= pd.DataFrame.to_numpy(df['Condition'])

In [18]:
X= np.delete(X,19,1)
X= np.delete(X,19,1)
X= np.delete(X,19,1)
X= np.delete(X,19,1)

In [19]:
print(headers)
headers.remove('Severity_Mild')
headers.remove('Severity_Moderate')
headers.remove('Severity_None')
headers.remove('Severity_Severe')
print(headers)

['Fever', 'Tiredness', 'Dry-Cough', 'Difficulty-in-Breathing', 'Sore-Throat', 'None_Sympton', 'Pains', 'Nasal-Congestion', 'Runny-Nose', 'Diarrhea', 'None_Experiencing', 'Age_0-9', 'Age_10-19', 'Age_20-24', 'Age_25-59', 'Age_60+', 'Gender_Female', 'Gender_Male', 'Gender_Transgender', 'Severity_Mild', 'Severity_Moderate', 'Severity_None', 'Severity_Severe', 'Contact_Dont-Know', 'Contact_No', 'Contact_Yes', 'Country', 'Condition']
['Fever', 'Tiredness', 'Dry-Cough', 'Difficulty-in-Breathing', 'Sore-Throat', 'None_Sympton', 'Pains', 'Nasal-Congestion', 'Runny-Nose', 'Diarrhea', 'None_Experiencing', 'Age_0-9', 'Age_10-19', 'Age_20-24', 'Age_25-59', 'Age_60+', 'Gender_Female', 'Gender_Male', 'Gender_Transgender', 'Contact_Dont-Know', 'Contact_No', 'Contact_Yes', 'Country', 'Condition']


## Chi Square

In [20]:
def get_max(x):
    max_val = 0
    for i in x:
        if i>max_val:
            max_val = i
    return max_val

In [21]:
def get_min(x):
    min_val = 99
    for i in x:
        if i<min_val:
            min_val = i
    return min_val

In [22]:
y = y.astype(np.int)

In [23]:
contingency_table = {}

for x_col in range(X.shape[1]):
    col = X[:,x_col]
    print(headers[x_col])
    
    for val_x in range(get_max(col)+1):
        count=0
        for val_y in range(get_max(y)+1):
            
            contingency_table[headers[x_col],val_x,val_y] = 0
                #count=0
                #if i == val:
                    #count = count+1

            #contingency_table[headers[x_col],val,] = [val, count]

Fever
Tiredness
Dry-Cough
Difficulty-in-Breathing
Sore-Throat
None_Sympton
Pains
Nasal-Congestion
Runny-Nose
Diarrhea
None_Experiencing
Age_0-9
Age_10-19
Age_20-24
Age_25-59
Age_60+
Gender_Female
Gender_Male
Gender_Transgender
Contact_Dont-Know
Contact_No
Contact_Yes
Country


In [24]:
print(contingency_table)

{('Fever', 0, 0): 0, ('Fever', 0, 1): 0, ('Fever', 0, 2): 0, ('Fever', 0, 3): 0, ('Fever', 1, 0): 0, ('Fever', 1, 1): 0, ('Fever', 1, 2): 0, ('Fever', 1, 3): 0, ('Tiredness', 0, 0): 0, ('Tiredness', 0, 1): 0, ('Tiredness', 0, 2): 0, ('Tiredness', 0, 3): 0, ('Tiredness', 1, 0): 0, ('Tiredness', 1, 1): 0, ('Tiredness', 1, 2): 0, ('Tiredness', 1, 3): 0, ('Dry-Cough', 0, 0): 0, ('Dry-Cough', 0, 1): 0, ('Dry-Cough', 0, 2): 0, ('Dry-Cough', 0, 3): 0, ('Dry-Cough', 1, 0): 0, ('Dry-Cough', 1, 1): 0, ('Dry-Cough', 1, 2): 0, ('Dry-Cough', 1, 3): 0, ('Difficulty-in-Breathing', 0, 0): 0, ('Difficulty-in-Breathing', 0, 1): 0, ('Difficulty-in-Breathing', 0, 2): 0, ('Difficulty-in-Breathing', 0, 3): 0, ('Difficulty-in-Breathing', 1, 0): 0, ('Difficulty-in-Breathing', 1, 1): 0, ('Difficulty-in-Breathing', 1, 2): 0, ('Difficulty-in-Breathing', 1, 3): 0, ('Sore-Throat', 0, 0): 0, ('Sore-Throat', 0, 1): 0, ('Sore-Throat', 0, 2): 0, ('Sore-Throat', 0, 3): 0, ('Sore-Throat', 1, 0): 0, ('Sore-Throat', 1, 1)

In [ ]:
for x_col in range(X.shape[1]):
    col = X[:,x_col]
    print(headers[x_col])
    
    for i in col:
        for j in y:
            contingency_table[headers[x_col],i,j] +=1
                

Fever


## Shuffle and Split

In [17]:
def shuffle_and_split(x,y):
    global x_train
    global x_test
    global y_train
    global y_test
    
    N = x.shape[0]
    P = np.arange(0,N,1)
    np.random.shuffle(P)
    lim= int(0.8*N)
    
    x_train = np.copy(x[P[0:lim],:])
    y_train = np.copy(y[P[0:lim]])
    x_test = np.copy(x[P[lim:N],:])
    y_test = np.copy(y[P[lim:N]])

In [18]:
shuffle_and_split(X,y)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(253440, 27)
(253440,)
(63360, 27)
(63360,)


In [19]:
print(x_train[0])

[0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 'No' 'No' 'No' '3' 1 0 0 3]


# Neural Network

## Activation Function Definitions

In [19]:
def sigmoid(x):
  
  #return 1/(1 + np.exp(-x))
  return np.tanh(x)

def sigmoid_prime(x):
  
  #return sigmoid(x)*(1 - sigmoid(x))
  return 1 - np.power(np.tanh(x), 2)

## Weights for NN

In [20]:
v = (2*np.random.random((2,3))-1)
w = (2*np.random.random((3,1))-1)

## Define the learning rate and the network structure

In [21]:
alpha = 0.1

number_input_units = 3
number_hidden_units = 2 # ignore bias term
number_output_units = 1

## Perform the feedforward and backpropagation to update the weights and train the network

In [22]:
error = []

for epoch in range(0, 2000):
    
  # randomly select a training example
  entry = np.random.randint(4)
    
  x = X[entry,:]

  # 1. Feedforward

  Z_in = []
  Z_out = []
    
  # Each hidden unit sums its weighted input signals
  for j in range(0, number_hidden_units):
    Z_in.append(np.dot(x, v[j,:]))
    Z_out.append(sigmoid(Z_in[j])) # and applies its activation function to computer its output
    
  
  # We append the bias term to layer_1
  Z_out = np.append(1, Z_out)
    
  Y_in = []
  Y_out = []
    
  # Each output unit sums its weighted input signals
  for k in range(0, number_output_units):
    Y_in.append(np.dot(Z_out, w))
    Y_out.append(sigmoid(Y_in[k]))
      
  Y_in = Y_in[0]
  Y_out = Y_out[0]
  
   
  # 2. Backpropagation

  delta_Y = (y[entry] - Y_out)*sigmoid_prime(Y_in)
    
  delta_Y = np.array(delta_Y)
   
  #err = y[entry]-Y_out
  err = np.power(y[entry]-Y_out, 2)
            
  error.append(err)
        
  delta_Z_in = []
  delta_Z = []
    
  # Each hidden unit sums its delta inputs from previous layer
  for j in range(0, number_hidden_units):
    delta_Z_in.append(delta_Y*w[j+1])
    delta_Z.append(delta_Z_in[j]*sigmoid_prime(Z_in[j]))
      
      
  delta_Z = np.array(delta_Z)
    
        
  # 3. Weights adjustment
           
  tmp = alpha*delta_Y*Z_out
     
  v = v + alpha*delta_Z*x
  w = np.add(w.T, tmp)
    
  w = w.T
         
    
fig = plt.figure()
plt.plot(error)
plt.ylabel('Error')
plt.xlabel('Iterations')
plt.show()

ValueError: shapes (28,) and (3,) not aligned: 28 (dim 0) != 3 (dim 0)

# Decision Tree

In [ ]:
height_values = []
acceleration_values = []
no_wheels_values=[]
width_values = []
length_values = []
fairing_values=[]
weight_values = []
max_speed_values = []
torque_values=[]
colour_values = []
#
classes = ['Yoshi','Golden_Mario','Luigi','Link','Mario','Shy_guy','Peachet','Wa_Luigi']
features = ['height','acceleration','no_wheels','width','length','fairing','weight','max_speed','torque','colour']

#
classes2code = {}
code2classes = {}
i=0
for v in classes:
    classes2code[v]=i
    code2classes[i]=v
    i+=1
#
#
# convert dataset to numpy arrays
n = len(features)  # number of features
m = len(data)  # number of instances
X = np.zeros([m,n],dtype=float)
Y = np.zeros([m],dtype=float)
for i in range(m):
    X[i,0] = data[i][0]
    X[i,1] = data[i][1]
    X[i,2] = data[i][2]
    X[i,3] = data[i][3]
    X[i,4] = data[i][4]
    X[i,5] = data[i][5]
    X[i,6] = data[i][6]
    X[i,7] = data[i][7]
    X[i,8] = data[i][8]
    X[i,9] = data[i][9]
    Y[i] = classes2code[data[i][10]]
    